# Making new discoveries in Hubble data
*Dr Michelle Lochner, Dr Bertie Seyffert & the ESA Datalabs team*

## Background
### Introduction
Our goal with this project is to use machine learning to make scientific discoveries in archived data. We're going to be focusing on galaxy images from the infamous Hubble Space Telescope. While all these images are beautiful, we're looking for anything unusual or weird that might warrant further investigation. In other words, we want to find anything that doesn't look like a "boring" spiral or elliptical galaxy (you can see below some examples of "boring" galaxies).

But why would we want to do this? Can't you just look at the data and find the interesting things? You can, as long as the dataset is small enough. The dataset we've put together for you has over 17000 images, which would take you hours or even days to look through. And this is a tiny subset of all the images that are in the Hubble archive, which number in the millions. As more powerful telescopes are being built, the data rates are just increasing and we need to turn to automated methods to help us make new discoveries.

### Resources
**Hubble and Galaxy Zoo** <br>
https://hubblesite.org/home <br>
https://www.zooniverse.org/projects/zookeeper/galaxy-zoo/ <br>
**General machine learning** <br>
https://www.youtube.com/watch?v=gLD8I3zcjB0 <br>
https://www.youtube.com/watch?v=ubhLmFQPgjc&t=1152s <br>
https://www.youtube.com/watch?v=g7caAxKBptc (much more advanced talk) <br>
**Convolutional neural networks and feature extraction** <br>
https://www.datacamp.com/tutorial/introduction-to-convolutional-neural-networks-cnns <br>
https://www.youtube.com/watch?v=RZvPgSiOYas (not fantastic but best I could find) <br>
**Local outlier factor** <br>
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.LocalOutlierFactor.html <br>
https://medium.com/@pramodch/understanding-lof-local-outlier-factor-for-implementation-1f6d4ff13ab9 <br>


### Import packages

In [189]:
import os
from pathlib import Path                                           # Used to handle filesystem paths
import pandas as pd                                                # Used to read in and work with the data as a "DataFrame"
import numpy as np                                                 # Our favourite numerical library
import matplotlib.pyplot as plt                                    # Our favourite plotting library
from sklearn.neighbors import LocalOutlierFactor                   # The suggested anomaly detection algorithm
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image                                              # This library will allow us to easily display png files in the notebook

### Read the relevant files

In [190]:
## The paths to the data and images directories in the team workspace
print('current working dir', os.getcwd())

## load data
data_directory = '/media/user/data/' 
images_directory = Path('/media/team_workspaces/astronomaly-project/images')

## The zoobot features after running PCA
features_reduced = pd.read_csv(data_directory + 'all-pca-components-refined.csv', index_col= 0)

## Coordinates for the sources
source_coords = pd.read_csv(data_directory + 'source-coordinates.csv', index_col= 1)

current working dir /media/user/notebooks


### Open the data

#### Each row of the DataFrame has a unique index which helps us keep track of exactly which set of numbers belongs to which source. 

In [191]:
features_reduced.head()

,SourceID,file_loc,feat_0_pca,feat_1_pca,feat_2_pca,feat_3_pca,feat_4_pca,feat_5_pca,feat_6_pca,feat_7_pca,feat_8_pca,feat_9_pca,feat_10_pca,feat_11_pca,feat_12_pca,feat_13_pca,feat_14_pca,feat_15_pca,feat_16_pca,feat_17_pca,feat_18_pca,feat_19_pca,feat_20_pca,feat_21_pca,feat_22_pca,feat_23_pca,feat_24_pca,feat_25_pca,feat_26_pca,feat_27_pca,feat_28_pca,feat_29_pca,feat_30_pca,feat_31_pca,feat_32_pca,feat_33_pca,feat_34_pca,feat_35_pca,feat_36_pca,feat_37_pca,feat_38_pca,feat_39_pca
0,AHZ2000ap3,/media/team_workspaces/astronomaly-project/ima...,0.644151,-0.607720,-0.652212,0.161491,0.329425,-0.111614,-0.078075,0.243963,0.225556,0.208291,-0.747757,0.040915,-0.173300,0.047926,0.121308,-0.088273,-0.024299,0.090302,-0.187438,-0.405518,-0.093745,-0.170736,0.235474,-0.089731,0.082089,-0.135348,0.041722,-0.006314,-0.056182,-0.058596,-0.047008,0.034110,0.045288,-0.014910,-0.027302,-0.006636,0.066010,0.091932,0.000969,0.081786
1,4000712889196,/media/team_workspaces/astronomaly-project/ima...,1.793052,-0.652613,-0.547502,-0.185649,0.189948,0.280023,-0.285091,0.526109,0.287551,0.019724,0.202724,0.030865,0.136582,0.071037,0.007827,0.084553,0.023075,-0.002175,-0.074796,0.017690,-0.098785,0.045782,0.065563,-0.062783,0.037058,0.042027,-0.040868,0.104335,-0.029692,-0.026881,-0.011317,0.019125,-0.010794,-0.032408,0.003039,-0.010592,0.056721,-0.019649,-0.013452,0.017829
2,4000710658615,/media/team_workspaces/astronomaly-project/ima...,1.944819,-0.582089,-0.348063,-0.240915,0.122469,0.270349,-0.146614,0.333893,0.026204,-0.147662,-0.135945,-0.014506,0.201758,0.036429,0.097208,0.184813,-0.027082,0.019965,-0.132242,-0.071320,-0.168242,0.033838,0.019874,0.050306,-0.013512,0.004749,-0.061095,0.063880,-0.010145,-0.025300,0.003306,0.047460,0.016832,-0.037377,0.016838,0.003930,0.026537,-0.018909,-0.034380,0.011928
3,AHZ2000ow7,/media/team_workspaces/astronomaly-project/ima...,-0.111912,-0.038260,0.540898,0.007854,0.466451,0.022677,-0.420133,0.039169,-0.212975,-0.124386,0.285441,-0.044144,0.097771,0.083764,-0.056121,-0.142532,-0.022763,-0.129097,0.040610,0.007049,0.031881,-0.014841,0.002715,0.029127,0.036973,-0.013699,-0.055546,0.009362,0.021781,-0.031641,0.072039,-0.012041,-0.008450,-0.006786,0.015151,0.015686,-0.020232,-0.018433,0.018422,-0.000777
4,4111276917,/media/team_workspaces/astronomaly-project/ima...,-1.876487,0.702289,0.362675,-0.520290,0.039717,-0.165041,-0.341243,0.122296,0.052079,0.026967,0.167456,-0.099207,-0.267685,-0.344153,0.153971,0.107811,-0.361514,-0.019896,-0.103078,-0.115273,-0.130121,-0.096965,0.095602,0.100216,-0.081191,0.070230,0.008064,0.179863,-0.038731,0.033161,-0.077778,0.000792,-0.052679,0.002063,-0.001525,-0.030409,-0.050910,-0.043423,0.009688,0.039821


## Preprocessing

In [192]:
## shape of the data
features_reduced.shape

(17390, 42)

In [193]:
## info about the data
features_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17390 entries, 0 to 17600
Data columns (total 42 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SourceID     17390 non-null  object 
 1   file_loc     17390 non-null  object 
 2   feat_0_pca   17390 non-null  float64
 3   feat_1_pca   17390 non-null  float64
 4   feat_2_pca   17390 non-null  float64
 5   feat_3_pca   17390 non-null  float64
 6   feat_4_pca   17390 non-null  float64
 7   feat_5_pca   17390 non-null  float64
 8   feat_6_pca   17390 non-null  float64
 9   feat_7_pca   17390 non-null  float64
 10  feat_8_pca   17390 non-null  float64
 11  feat_9_pca   17390 non-null  float64
 12  feat_10_pca  17390 non-null  float64
 13  feat_11_pca  17390 non-null  float64
 14  feat_12_pca  17390 non-null  float64
 15  feat_13_pca  17390 non-null  float64
 16  feat_14_pca  17390 non-null  float64
 17  feat_15_pca  17390 non-null  float64
 18  feat_16_pca  17390 non-null  float64
 19  feat

In [194]:
print(features_reduced.dtypes)

SourceID        object
file_loc        object
feat_0_pca     float64
feat_1_pca     float64
feat_2_pca     float64
feat_3_pca     float64
feat_4_pca     float64
feat_5_pca     float64
feat_6_pca     float64
feat_7_pca     float64
feat_8_pca     float64
feat_9_pca     float64
feat_10_pca    float64
feat_11_pca    float64
feat_12_pca    float64
feat_13_pca    float64
feat_14_pca    float64
feat_15_pca    float64
feat_16_pca    float64
feat_17_pca    float64
feat_18_pca    float64
feat_19_pca    float64
feat_20_pca    float64
feat_21_pca    float64
feat_22_pca    float64
feat_23_pca    float64
feat_24_pca    float64
feat_25_pca    float64
feat_26_pca    float64
feat_27_pca    float64
feat_28_pca    float64
feat_29_pca    float64
feat_30_pca    float64
feat_31_pca    float64
feat_32_pca    float64
feat_33_pca    float64
feat_34_pca    float64
feat_35_pca    float64
feat_36_pca    float64
feat_37_pca    float64
feat_38_pca    float64
feat_39_pca    float64
dtype: object


###

In [195]:
## check missing values
features_reduced.isna()

,SourceID,file_loc,feat_0_pca,feat_1_pca,feat_2_pca,feat_3_pca,feat_4_pca,feat_5_pca,feat_6_pca,feat_7_pca,feat_8_pca,feat_9_pca,feat_10_pca,feat_11_pca,feat_12_pca,feat_13_pca,feat_14_pca,feat_15_pca,feat_16_pca,feat_17_pca,feat_18_pca,feat_19_pca,feat_20_pca,feat_21_pca,feat_22_pca,feat_23_pca,feat_24_pca,feat_25_pca,feat_26_pca,feat_27_pca,feat_28_pca,feat_29_pca,feat_30_pca,feat_31_pca,feat_32_pca,feat_33_pca,feat_34_pca,feat_35_pca,feat_36_pca,feat_37_pca,feat_38_pca,feat_39_pca
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17596,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17597,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17598,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17599,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [196]:
features_reduced.isna().sum().sum()

0

### No missing values on dataset

### We can check and study the second data frame, `source_coords` - position data frame

In [197]:
## 
source_coords.head()

,Unnamed: 0,RA,Dec,file_loc
SourceID,,,,
AHZ2000ap3,0,150.3053182,1.8976633,/media/team_workspaces/astronomaly-project/ima...
4000712889196,1,11.4864402,41.9962016,/media/team_workspaces/astronomaly-project/ima...
4000710658615,2,11.4642217,42.1513209,/media/team_workspaces/astronomaly-project/ima...
AHZ2000ow7,3,150.53574609999998,2.1374021,/media/team_workspaces/astronomaly-project/ima...
4111276917,4,344.677271,-34.824124299999994,/media/team_workspaces/astronomaly-project/ima...


In [198]:
## shape
source_coords.shape

(17601, 4)

In [199]:
source_coords.dtypes

Unnamed: 0     int64
RA            object
Dec           object
file_loc      object
dtype: object

### we can drop unnamed column/axis1 `Unnamed: 0`

In [200]:
source_coords = source_coords.drop('Unnamed: 0', axis=1)

In [201]:
source_coords.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17601 entries, AHZ2000ap3 to COS_2333
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   RA        15782 non-null  object
 1   Dec       15782 non-null  object
 2   file_loc  17601 non-null  object
dtypes: object(3)
memory usage: 550.0+ KB


####

### we can see that features are mixed up with information about the source like its name and where the file is stored. For algorithm simplicity we need to drop them. we can do so by spliting the data set with its copy

In [202]:
## a copy of the two dataframes
features = features_reduced.copy()
positions = source_coords.copy()

##

In [203]:
features.head()

,SourceID,file_loc,feat_0_pca,feat_1_pca,feat_2_pca,feat_3_pca,feat_4_pca,feat_5_pca,feat_6_pca,feat_7_pca,feat_8_pca,feat_9_pca,feat_10_pca,feat_11_pca,feat_12_pca,feat_13_pca,feat_14_pca,feat_15_pca,feat_16_pca,feat_17_pca,feat_18_pca,feat_19_pca,feat_20_pca,feat_21_pca,feat_22_pca,feat_23_pca,feat_24_pca,feat_25_pca,feat_26_pca,feat_27_pca,feat_28_pca,feat_29_pca,feat_30_pca,feat_31_pca,feat_32_pca,feat_33_pca,feat_34_pca,feat_35_pca,feat_36_pca,feat_37_pca,feat_38_pca,feat_39_pca
0,AHZ2000ap3,/media/team_workspaces/astronomaly-project/ima...,0.644151,-0.607720,-0.652212,0.161491,0.329425,-0.111614,-0.078075,0.243963,0.225556,0.208291,-0.747757,0.040915,-0.173300,0.047926,0.121308,-0.088273,-0.024299,0.090302,-0.187438,-0.405518,-0.093745,-0.170736,0.235474,-0.089731,0.082089,-0.135348,0.041722,-0.006314,-0.056182,-0.058596,-0.047008,0.034110,0.045288,-0.014910,-0.027302,-0.006636,0.066010,0.091932,0.000969,0.081786
1,4000712889196,/media/team_workspaces/astronomaly-project/ima...,1.793052,-0.652613,-0.547502,-0.185649,0.189948,0.280023,-0.285091,0.526109,0.287551,0.019724,0.202724,0.030865,0.136582,0.071037,0.007827,0.084553,0.023075,-0.002175,-0.074796,0.017690,-0.098785,0.045782,0.065563,-0.062783,0.037058,0.042027,-0.040868,0.104335,-0.029692,-0.026881,-0.011317,0.019125,-0.010794,-0.032408,0.003039,-0.010592,0.056721,-0.019649,-0.013452,0.017829
2,4000710658615,/media/team_workspaces/astronomaly-project/ima...,1.944819,-0.582089,-0.348063,-0.240915,0.122469,0.270349,-0.146614,0.333893,0.026204,-0.147662,-0.135945,-0.014506,0.201758,0.036429,0.097208,0.184813,-0.027082,0.019965,-0.132242,-0.071320,-0.168242,0.033838,0.019874,0.050306,-0.013512,0.004749,-0.061095,0.063880,-0.010145,-0.025300,0.003306,0.047460,0.016832,-0.037377,0.016838,0.003930,0.026537,-0.018909,-0.034380,0.011928
3,AHZ2000ow7,/media/team_workspaces/astronomaly-project/ima...,-0.111912,-0.038260,0.540898,0.007854,0.466451,0.022677,-0.420133,0.039169,-0.212975,-0.124386,0.285441,-0.044144,0.097771,0.083764,-0.056121,-0.142532,-0.022763,-0.129097,0.040610,0.007049,0.031881,-0.014841,0.002715,0.029127,0.036973,-0.013699,-0.055546,0.009362,0.021781,-0.031641,0.072039,-0.012041,-0.008450,-0.006786,0.015151,0.015686,-0.020232,-0.018433,0.018422,-0.000777
4,4111276917,/media/team_workspaces/astronomaly-project/ima...,-1.876487,0.702289,0.362675,-0.520290,0.039717,-0.165041,-0.341243,0.122296,0.052079,0.026967,0.167456,-0.099207,-0.267685,-0.344153,0.153971,0.107811,-0.361514,-0.019896,-0.103078,-0.115273,-0.130121,-0.096965,0.095602,0.100216,-0.081191,0.070230,0.008064,0.179863,-0.038731,0.033161,-0.077778,0.000792,-0.052679,0.002063,-0.001525,-0.030409,-0.050910,-0.043423,0.009688,0.039821


### Separate the features and the metadata(crucial to understanding data but not for the ML) about each source

In [204]:
## remove feature's id, location
features = features.drop(columns= ['SourceID', 'file_loc'], axis= 1)
metadata = features_reduced[['SourceID', 'file_loc']]

In [205]:
metadata

,SourceID,file_loc
0,AHZ2000ap3,/media/team_workspaces/astronomaly-project/ima...
1,4000712889196,/media/team_workspaces/astronomaly-project/ima...
2,4000710658615,/media/team_workspaces/astronomaly-project/ima...
3,AHZ2000ow7,/media/team_workspaces/astronomaly-project/ima...
4,4111276917,/media/team_workspaces/astronomaly-project/ima...
...,...,...
17596,64189308,/media/team_workspaces/astronomaly-project/ima...
17597,4000709888007,/media/team_workspaces/astronomaly-project/ima...
17598,AHZ20013on,/media/team_workspaces/astronomaly-project/ima...
17599,AHZ2001ivx,/media/team_workspaces/astronomaly-project/ima...


In [206]:
features

,feat_0_pca,feat_1_pca,feat_2_pca,feat_3_pca,feat_4_pca,feat_5_pca,feat_6_pca,feat_7_pca,feat_8_pca,feat_9_pca,feat_10_pca,feat_11_pca,feat_12_pca,feat_13_pca,feat_14_pca,feat_15_pca,feat_16_pca,feat_17_pca,feat_18_pca,feat_19_pca,feat_20_pca,feat_21_pca,feat_22_pca,feat_23_pca,feat_24_pca,feat_25_pca,feat_26_pca,feat_27_pca,feat_28_pca,feat_29_pca,feat_30_pca,feat_31_pca,feat_32_pca,feat_33_pca,feat_34_pca,feat_35_pca,feat_36_pca,feat_37_pca,feat_38_pca,feat_39_pca
0,0.644151,-0.607720,-0.652212,0.161491,0.329425,-0.111614,-0.078075,0.243963,0.225556,0.208291,-0.747757,0.040915,-0.173300,0.047926,0.121308,-0.088273,-0.024299,0.090302,-0.187438,-0.405518,-0.093745,-0.170736,0.235474,-0.089731,0.082089,-0.135348,0.041722,-0.006314,-0.056182,-0.058596,-0.047008,0.034110,0.045288,-0.014910,-0.027302,-0.006636,0.066010,0.091932,0.000969,0.081786
1,1.793052,-0.652613,-0.547502,-0.185649,0.189948,0.280023,-0.285091,0.526109,0.287551,0.019724,0.202724,0.030865,0.136582,0.071037,0.007827,0.084553,0.023075,-0.002175,-0.074796,0.017690,-0.098785,0.045782,0.065563,-0.062783,0.037058,0.042027,-0.040868,0.104335,-0.029692,-0.026881,-0.011317,0.019125,-0.010794,-0.032408,0.003039,-0.010592,0.056721,-0.019649,-0.013452,0.017829
2,1.944819,-0.582089,-0.348063,-0.240915,0.122469,0.270349,-0.146614,0.333893,0.026204,-0.147662,-0.135945,-0.014506,0.201758,0.036429,0.097208,0.184813,-0.027082,0.019965,-0.132242,-0.071320,-0.168242,0.033838,0.019874,0.050306,-0.013512,0.004749,-0.061095,0.063880,-0.010145,-0.025300,0.003306,0.047460,0.016832,-0.037377,0.016838,0.003930,0.026537,-0.018909,-0.034380,0.011928
3,-0.111912,-0.038260,0.540898,0.007854,0.466451,0.022677,-0.420133,0.039169,-0.212975,-0.124386,0.285441,-0.044144,0.097771,0.083764,-0.056121,-0.142532,-0.022763,-0.129097,0.040610,0.007049,0.031881,-0.014841,0.002715,0.029127,0.036973,-0.013699,-0.055546,0.009362,0.021781,-0.031641,0.072039,-0.012041,-0.008450,-0.006786,0.015151,0.015686,-0.020232,-0.018433,0.018422,-0.000777
4,-1.876487,0.702289,0.362675,-0.520290,0.039717,-0.165041,-0.341243,0.122296,0.052079,0.026967,0.167456,-0.099207,-0.267685,-0.344153,0.153971,0.107811,-0.361514,-0.019896,-0.103078,-0.115273,-0.130121,-0.096965,0.095602,0.100216,-0.081191,0.070230,0.008064,0.179863,-0.038731,0.033161,-0.077778,0.000792,-0.052679,0.002063,-0.001525,-0.030409,-0.050910,-0.043423,0.009688,0.039821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17596,-1.596677,-0.390882,-0.305283,-0.438172,-1.094033,0.054087,0.247124,0.156970,-0.447714,-0.183841,-0.119676,-0.124356,0.181530,0.132627,0.147674,-0.223080,0.127351,0.179608,0.173148,-0.113438,-0.302634,-0.097667,0.452911,0.010397,0.234178,0.146059,0.032625,-0.145444,-0.002630,0.073853,0.081421,-0.066399,-0.021305,0.088554,0.064309,-0.076932,0.009420,-0.044328,-0.101527,0.013420
17597,-0.199175,-0.230402,-0.786815,-0.264246,-0.084064,-0.182996,0.337683,1.069785,0.148960,-0.381705,0.399367,0.138373,0.097432,0.098849,0.306672,-0.285690,-0.053076,-0.071591,-0.001020,0.107713,0.103741,-0.135786,-0.017720,-0.044992,-0.089225,0.090047,0.085491,0.039596,0.004893,0.115876,0.052100,0.082204,-0.014934,0.014840,-0.024947,0.062010,0.095045,-0.071596,0.005432,0.025155
17598,-0.626084,0.242955,0.818994,-0.090439,-0.500461,0.241786,-0.386057,0.056521,-0.111743,-0.097945,-0.148672,0.267963,-0.036391,-0.029762,0.066132,-0.146546,-0.208761,0.071067,-0.115876,0.095833,0.000155,-0.025722,0.066078,-0.089437,-0.018925,0.077774,-0.027524,0.010073,0.104444,0.006854,0.035144,-0.016096,-0.027457,-0.020181,0.036698,-0.037808,0.016292,0.010604,0.010578,0.030527
17599,-0.492462,-1.114859,0.558716,0.376411,0.304337,-0.003205,-0.300885,0.091722,-0.281236,-0.547345,-0.033521,-0.114167,-0.200025,0.163555,-0.123838,-0.144190,-0.138288,0.168889,0.001023,-0.026754,0.063589,0.149431,0.070321,0.103500,0.000594,-0.033076,0.058839,-0.013738,-0.082424,-0.069562,0.068283,

In [207]:
features.shape

(17390, 40)

In [208]:
features.dtypes

feat_0_pca     float64
feat_1_pca     float64
feat_2_pca     float64
feat_3_pca     float64
feat_4_pca     float64
feat_5_pca     float64
feat_6_pca     float64
feat_7_pca     float64
feat_8_pca     float64
feat_9_pca     float64
feat_10_pca    float64
feat_11_pca    float64
feat_12_pca    float64
feat_13_pca    float64
feat_14_pca    float64
feat_15_pca    float64
feat_16_pca    float64
feat_17_pca    float64
feat_18_pca    float64
feat_19_pca    float64
feat_20_pca    float64
feat_21_pca    float64
feat_22_pca    float64
feat_23_pca    float64
feat_24_pca    float64
feat_25_pca    float64
feat_26_pca    float64
feat_27_pca    float64
feat_28_pca    float64
feat_29_pca    float64
feat_30_pca    float64
feat_31_pca    float64
feat_32_pca    float64
feat_33_pca    float64
feat_34_pca    float64
feat_35_pca    float64
feat_36_pca    float64
feat_37_pca    float64
feat_38_pca    float64
feat_39_pca    float64
dtype: object

### Everything is now numerical, which is good for ML algorithms

In [209]:
features.isna().sum().sum()

0

## Local Outlier Factor(LOF) algorithm


### We can use the attribute `negative_outlier_factor_` to order the data from most to least anomous. This is a slightly different way to use lof algorithm, as it's designed to tell you what's an anomaly and what isn't but we're interested in what's the *most* anomalous sources.

In [210]:
# to run LOF
lof = LocalOutlierFactor(n_neighbors= 10)     # choosing the parameter n-neibours, is reasable to choose 9 since our data is large (17390, 40)

In [211]:
# predictions and score
pred = lof.fit_predict(features)               # features will be the inputs
lof_score = lof.negative_outlier_factor_      # outliers tend to have a larger LOF_score and normalies will have negative_outlier_factor_ close to -1

In [212]:
print('The negative Outlier score:', lof_score)

#print('\nScores:', lof.negative_outlier_factor_)

The negative Outlier score: [-1.27294177 -1.24814422 -1.11408177 ... -1.00345454 -1.02030087
 -1.17622586]


## Visualise the results

### Random examples
First we plot some random examples so we can compare

In [213]:
#random_images = np.random.choice(list(features_reduced.file_loc), replace=False, size=20)
#plt.figure(figsize=(8,10))
#for cnt, image in enumerate(random_images, start=1):
#    im = Image.open(image)
#    plt.subplot(5,4,cnt)
#    plt.imshow(im)
#    plt.axis('off');
#plt.tight_layout()

### Most anomalous sources
Use the `negative_outlier_factor_` attribute of LOF to find the highest-scoring objects and plot them.

In [214]:
# Code to plot the top 20 high scoring anomalies